In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM, Bidirectional, SimpleRNN
from tensorflow.keras.layers import GlobalAveragePooling1D, SpatialDropout1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report

In [2]:
# load data and then encode the labels into numeric values using LabelEncoder

In [3]:
data=pd.read_csv("Text_pre.csv")

In [4]:
data['label_encoded'] = LabelEncoder().fit_transform(data['label'])

In [5]:
# splits the data into training and testing sets
X = data['cleaned_text']
y = data['label_encoded']
X = data['cleaned_text'].astype(str)  # Convert to string to handle any float or NaN issues
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
#  Text Preprocessing with Tokenization and Padding
vocab_size = 10000
max_length = 100
embedding_dim = 100

tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)
X_train_pad = pad_sequences(X_train_seq, maxlen=max_length, padding='post', truncating='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_length, padding='post', truncating='post')


In [7]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Bidirectional, SpatialDropout1D
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.metrics import Precision, Recall
from sklearn.metrics import classification_report

# Model building function
def build_model(model_type="LSTM", vocab_size=10000, embedding_dim=100, max_length=100):
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_dim, input_length=max_length))
    model.add(SpatialDropout1D(0.2))
    
    if model_type == "BiLSTM":
        model.add(Bidirectional(LSTM(64, return_sequences=False)))
    else:
        model.add(LSTM(64, return_sequences=False))  # Default to LSTM if not BiLSTM
    
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # Binary classification

    model.compile(optimizer=Adam(learning_rate=0.001), 
                  loss='binary_crossentropy', 
                  metrics=['accuracy', Precision(), Recall()])
    return model

# Training and evaluation function
def train_and_evaluate(model_type, X_train_pad, y_train, X_test_pad, y_test):
    model = build_model(model_type)
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    
    # Train the model
    history = model.fit(X_train_pad, y_train, 
                        epochs=10, 
                        batch_size=64, 
                        validation_split=0.2,
                        callbacks=[early_stopping])
    
    # Evaluation of the model
    y_pred = (model.predict(X_test_pad) > 0.5).astype("int32")
    
    # Print classification report
    print(f"Classification Report for {model_type}:")
    print(classification_report(y_test, y_pred, target_names=['CG', 'OR']))

# Loop through and train and evaluate LSTM and BiLSTM models
# Assuming X_train_pad, y_train, X_test_pad, y_test are already defined
for model_type in ["LSTM", "BiLSTM"]:
    train_and_evaluate(model_type, X_train_pad, y_train, X_test_pad, y_test)


Epoch 1/10


C:\Users\Admin\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


405/405 ━━━━━━━━━━━━━━━━━━━━ 52s 111ms/step - accuracy: 0.5798 - loss: 0.6671 - precision: 0.6838 - recall: 0.2903 - val_accuracy: 0.6738 - val_loss: 0.6248 - val_precision: 0.7454 - val_recall: 0.5319
Epoch 2/10
405/405 ━━━━━━━━━━━━━━━━━━━━ 86s 122ms/step - accuracy: 0.6679 - loss: 0.6298 - precision: 0.7034 - recall: 0.5728 - val_accuracy: 0.6802 - val_loss: 0.6003 - val_precision: 0.7110 - val_recall: 0.6113
Epoch 3/10
405/405 ━━━━━━━━━━━━━━━━━━━━ 80s 116ms/step - accuracy: 0.7818 - loss: 0.4813 - precision: 0.8395 - recall: 0.6975 - val_accuracy: 0.8621 - val_loss: 0.3384 - val_precision: 0.8945 - val_recall: 0.8223
Epoch 4/10
405/405 ━━━━━━━━━━━━━━━━━━━━ 44s 109ms/step - accuracy: 0.8873 - loss: 0.2739 - precision: 0.9016 - recall: 0.8704 - val_accuracy: 0.9009 - val_loss: 0.2392 - val_precision: 0.8934 - val_recall: 0.9113
Epoch 5/10
405/405 ━━━━━━━━━━━━━━━━━━━━ 47s 116ms/step - accuracy: 0.9312 - loss: 0.1779 - precision: 0.9325 - recall: 0.9279 - val_accuracy: 0.9026 - val_loss

C:\Users\Admin\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


405/405 ━━━━━━━━━━━━━━━━━━━━ 86s 187ms/step - accuracy: 0.7694 - loss: 0.4393 - precision_1: 0.8079 - recall_1: 0.6773 - val_accuracy: 0.8983 - val_loss: 0.2343 - val_precision_1: 0.8712 - val_recall_1: 0.9356
Epoch 2/10
405/405 ━━━━━━━━━━━━━━━━━━━━ 68s 168ms/step - accuracy: 0.9266 - loss: 0.1847 - precision_1: 0.9240 - recall_1: 0.9295 - val_accuracy: 0.8938 - val_loss: 0.2472 - val_precision_1: 0.8428 - val_recall_1: 0.9692
Epoch 3/10
405/405 ━━━━━━━━━━━━━━━━━━━━ 63s 156ms/step - accuracy: 0.9413 - loss: 0.1424 - precision_1: 0.9392 - recall_1: 0.9429 - val_accuracy: 0.9124 - val_loss: 0.2108 - val_precision_1: 0.9143 - val_recall_1: 0.9107
Epoch 4/10
405/405 ━━━━━━━━━━━━━━━━━━━━ 60s 147ms/step - accuracy: 0.9644 - loss: 0.0916 - precision_1: 0.9661 - recall_1: 0.9621 - val_accuracy: 0.9136 - val_loss: 0.2396 - val_precision_1: 0.9153 - val_recall_1: 0.9122
Epoch 5/10
405/405 ━━━━━━━━━━━━━━━━━━━━ 59s 145ms/step - accuracy: 0.9731 - loss: 0.0715 - precision_1: 0.9751 - recall_1: 0.97

In [8]:
#Model Training, Prediction, and Evaluation Using BERT for Sequence Classification
#fine-tune a BERT model for sequence classification using the Transformers library by Hugging Face

In [9]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from tensorflow.keras.optimizers import Adam

In [10]:
pip install accelerate

Note: you may need to restart the kernel to use updated packages.


In [11]:
pip install accelerate>=0.26.0

Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124

Looking in indexes: https://download.pytorch.org/whl/cu124
Note: you may need to restart the kernel to use updated packages.


In [13]:
pip install transformers[torch]

Note: you may need to restart the kernel to use updated packages.


In [16]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Check if CUDA (GPU) is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Print the device being used (CPU or GPU)
print(f"Using device: {device}")

# Load dataset and preprocess
data = pd.read_csv('Text_pre.csv')  # Replace with your file path
data['label_encoded'] = LabelEncoder().fit_transform(data['label'])

# Split dataset
X = data['cleaned_text'].astype(str)
y = data['label_encoded']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Custom Dataset class to handle text and labels
class ReviewDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts.iloc[idx]
        label = self.labels.iloc[idx]
        encoding = self.tokenizer(
            text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten().to(device),
            'attention_mask': encoding['attention_mask'].flatten().to(device),
            'labels': torch.tensor(label, dtype=torch.long).to(device)
        }

# Create dataset and data loaders
train_dataset = ReviewDataset(X_train, y_train, tokenizer)
test_dataset = ReviewDataset(X_test, y_test, tokenizer)

# Load pre-trained BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2).to(device)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',# Adjust evaluation strategy (can set to None to avoid evaluation)
    per_device_train_batch_size=4,# 
    per_device_eval_batch_size=4,
    num_train_epochs=2,  # Consider reducing epochs for faster traini
    weight_decay=0.01,
    logging_dir='./logs',
)

# Define Trainer for training BERT model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer
)

# Train model
trainer.train()

# Evaluate on test set
predictions = trainer.predict(test_dataset)
pred_labels = torch.argmax(torch.tensor(predictions.predictions), axis=1).numpy()

from sklearn.metrics import classification_report
print(classification_report(y_test, pred_labels, target_names=['CG', 'OR']))

Using device: cpu


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\Admin\anaconda3\Lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\Admin\AppData\Local\Temp\ipykernel_6024\3332937762.py:72: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,0.409800,0.385804
2,0.277100,0.357262


              precision    recall  f1-score   support

          CG       0.89      0.95      0.92      4016
          OR       0.94      0.89      0.91      4071

    accuracy                           0.92      8087
   macro avg       0.92      0.92      0.92      8087
weighted avg       0.92      0.92      0.92      8087



In [17]:
## saving  trained model, tokenizer, and training arguments
import os
import json

# Define a directory to save the model and tokenizer
model_dir = 'BERT./saved_model'
os.makedirs(model_dir, exist_ok=True)

# Save the trained BERT model
model.save_pretrained(model_dir)

# Save the tokenizer
tokenizer.save_pretrained(model_dir)

# Save training arguments as a JSON file
with open(os.path.join(model_dir, 'training_args.json'), 'w') as f:
    json.dump(training_args.to_dict(), f)

print(f"Model, tokenizer, and training arguments saved in {model_dir}")


Model, tokenizer, and training arguments saved in BERT./saved_model


In [ ]:
#Conclusion:
The model is highly effective at detecting fake reviews with strong performance across both classes. With an accuracy of 92%, it strikes a good balance between precision and recall, showing that it is both accurate and reliable in classifying reviews as genuine or fake. This result is promising for the task of fake review detection.
 